In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import response_function_tools_22_Feb
from scipy.fft import rfft, fftfreq, irfft

In [ ]:
# Create the transfer functions at 1 kHz (out to 500 Hz positive)
freqs = np.arange(1, 500, 1)
avg_tf_smooth, avg_tf_raw = response_function_tools_22_Feb.generate_response_function_from_netcdf_file('/Users/mawa7160/dev/data/LIBERA/Calibration/22_Oct/libera_raw_detector_1024_1227.nc', freqs)
ceres_response = response_function_tools_22_Feb.libera_get_ceres_frequency_response(freqs)

on_orbit_factor = np.abs(avg_tf_smooth)/np.abs(avg_tf_raw)
ceres_factor = np.abs(ceres_response)/np.abs(avg_tf_smooth)

In [ ]:
# Generate a point spread function
# Sampled at 1 kHz for 1 second with a 20.8 millisecond high (at 1 kHz will be 21 ms
# Using estimated milliwatt estimates from detector data low is 312 mW and high is 318 mW
point_spread_function = np.ones(1000)*312
point_spread_function[488:509] = point_spread_function[488:509]+6
time = np.arange(0,1, 0.001)
plt.plot(time,point_spread_function)

In [ ]:
# On orbit smoothing done via fft
full_freq = fftfreq(1000, 1 / 1000)
pos_index = np.where(full_freq>0)
pos_freq = full_freq[pos_index]

signal_fft = rfft(point_spread_function)
#plt.plot(pos_freq, np.abs(signal_fft[pos_index]))
trimmed = signal_fft[pos_index]
smoothed = trimmed*on_orbit_factor
front_added = np.insert(smoothed, 0, signal_fft[0])
full = np.append(front_added, signal_fft[-1])
smoothed_signal_ifft = irfft(full)
plt.plot(smoothed_signal_ifft)
print(smoothed_signal_ifft.max())

In [ ]:
#Using from above
smoothed_data = smoothed_signal_ifft

# Test 1
# Direct conversion to CERES at 1 kHz rate
# On orbit smoothing done via fft
full_freq = fftfreq(1000, 1 / 1000)
pos_index = np.where(full_freq>0)
pos_freq = full_freq[pos_index]

signal_fft = rfft(smoothed_data)
#plt.plot(pos_freq, np.abs(signal_fft[pos_index]))
trimmed = signal_fft[pos_index]
smoothed = trimmed*ceres_factor
front_added = np.insert(smoothed, 0, signal_fft[0])
full = np.append(front_added, signal_fft[-1])
processed_ceres_matched_signal_ifft = irfft(full)
plt.plot(processed_ceres_matched_signal_ifft)

In [ ]:
# Create smoothing factors at 200 Hz (out to 100 Hz positive)
freqs_100hz = np.arange(1, 100, 1)
avg_tf_smooth_100hz, avg_tf_raw_100hz = response_function_tools_22_Feb.generate_response_function_from_netcdf_file('/Users/mawa7160/dev/data/LIBERA/Calibration/22_Oct/libera_raw_detector_1024_1227.nc', freqs_100hz)
ceres_response_100hz = response_function_tools_22_Feb.libera_get_ceres_frequency_response(freqs_100hz)

on_orbit_factor_100hz = np.abs(avg_tf_smooth_100hz)/np.abs(avg_tf_raw_100hz)
ceres_factor_100hz = np.abs(ceres_response_100hz)/np.abs(avg_tf_smooth_100hz)
plt.semilogx(freqs_100hz, np.abs(ceres_response_100hz), '-xg')
plt.semilogx(freqs_100hz, np.abs(avg_tf_smooth_100hz),'-xb')

In [ ]:
# Test 2 Downsampling to 200 Hz
smoothed_downsampled_data = smoothed_signal_ifft[0:-1:5]
downsampled_time = time[0:-1:5]
#print(smoothed_downsampled_data.max())
#plt.plot(downsampled_time, smoothed_downsampled_data)
full_freq = fftfreq(200, 1 / 200)
pos_index = np.where(full_freq>0)
pos_freq = full_freq[pos_index]

signal_fft = rfft(smoothed_downsampled_data)
#plt.plot(pos_freq, np.abs(signal_fft[pos_index]))
trimmed = signal_fft[pos_index]
smoothed = trimmed*ceres_factor_100hz
front_added = np.insert(smoothed, 0, signal_fft[0])
full = np.append(front_added, signal_fft[-1])
processed_ceres_matched_signal_ifft_200hz = irfft(full)
plt.plot(processed_ceres_matched_signal_ifft_200hz)
print(processed_ceres_matched_signal_ifft_200hz.max())

In [ ]:
# Test 3 Downsampling to 200 Hz before smoothing
downsampled_data = point_spread_function[0:-1:5]
downsampled_time = time[0:-1:5]
#plt.plot(downsampled_time, downsampled_data)
full_freq = fftfreq(200, 1 / 200)
pos_index = np.where(full_freq>0)
pos_freq = full_freq[pos_index]

signal_fft = rfft(downsampled_data)
#plt.plot(pos_freq, np.abs(signal_fft[pos_index]))
trimmed = signal_fft[pos_index]
smoothed = trimmed*ceres_factor_100hz*on_orbit_factor_100hz
front_added = np.insert(smoothed, 0, signal_fft[0])
full = np.append(front_added, signal_fft[-1])
processed_ceres_matched_signal_ifft_200hz = irfft(full)
plt.plot(processed_ceres_matched_signal_ifft_200hz)
print(processed_ceres_matched_signal_ifft_200hz.max())